In [ ]:
https://js.langchain.com/docs/modules/model_io/models/chat/how_to/function_calling

# OpenAI Function calling

함수 호출은 LLM에서 다양한 목적으로 구조화된 출력을 얻는 유용한 방법입니다. "함수"에 대한 스키마를 제공함으로써 LLM은 하나를 선택하고 해당 스키마에 맞는 응답을 출력하려고 노력할 것입니다.

이름이 LLM이 실제로 코드를 실행하고 함수를 호출한다는 것을 시사하고 있지만, 더 정확히 말하면 LLM은 가상 함수가 사용할 인수의 스키마와 일치하는 매개변수를 채우고 있습니다. 이러한 구조화된 응답은 우리가 원하는 목적으로 사용할 수 있습니다!

함수 호출은 LangChain에서 OpenAI Functions 에이전트와 구조화된 출력 체인을 포함한 여러 인기있는 기능의 기본 구성 요소로 작용합니다. 이러한 더 구체적인 사용 사례 외에도 함수 매개변수를 직접 모델에 첨부하고 호출할 수 있습니다.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

In [2]:
function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}


llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})


response = response.additional_kwargs["function_call"]["arguments"]

response

'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"476 AD","correct":false},{"answer":"100 AD","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What event marked the fall of the Western Roman Empire?","answers":[{"answer":"Sack of Rome","correct":false},{"answer":"Battle of Adrianople","correct":false},{"answer":"476 AD","correct":true}]}]}'

In [3]:
import json

for question in json.loads(response)["questions"]:
    print(question)

{'question': 'What year was Rome founded?', 'answers': [{'answer': '753 BC', 'correct': True}, {'answer': '476 AD', 'correct': False}, {'answer': '100 AD', 'correct': False}]}
{'question': 'Who was the first emperor of Rome?', 'answers': [{'answer': 'Julius Caesar', 'correct': False}, {'answer': 'Augustus', 'correct': True}, {'answer': 'Nero', 'correct': False}]}
{'question': 'What event marked the fall of the Western Roman Empire?', 'answers': [{'answer': 'Sack of Rome', 'correct': False}, {'answer': 'Battle of Adrianople', 'correct': False}, {'answer': '476 AD', 'correct': True}]}
